In [1]:
import base as b
import pandas as pd
import datetime as dt
from typing import Dict
import talib as ta
import utils as ut
from constants import *
from logger_settings import logger

settings = {
    "small_sma": 3,
    "big_sma": 10,
}

instrument = b.Instrument(name="NIFTY 22650 CALL 7 Mar 2024")

SYMBOL = "NIFTY"
expiry = dt.datetime.strptime("2024-03-21", "%Y-%m-%d").date()
otype = "CE"
strike = 22000
date = dt.datetime.strptime("2024-03-15", "%Y-%m-%d").date()

tdf = ut.get_ticks(symbol=SYMBOL, expiry=expiry, strike=strike, otype=otype, date=date)

In [16]:
tdf.index

DatetimeIndex(['2024-03-15 09:15:02', '2024-03-15 09:15:03',
               '2024-03-15 09:15:03', '2024-03-15 09:15:04',
               '2024-03-15 09:15:05', '2024-03-15 09:15:05',
               '2024-03-15 09:15:06', '2024-03-15 09:15:07',
               '2024-03-15 09:15:08', '2024-03-15 09:15:08',
               ...
               '2024-03-15 15:29:54', '2024-03-15 15:29:55',
               '2024-03-15 15:29:55', '2024-03-15 15:29:56',
               '2024-03-15 15:29:57', '2024-03-15 15:29:58',
               '2024-03-15 15:29:58', '2024-03-15 15:29:59',
               '2024-03-15 15:29:59', '2024-03-15 15:29:59'],
              dtype='datetime64[ns]', name='last_trade_time', length=33885, freq=None)

In [4]:
class Phase(b.BasePhase):
    def __init__(self, initiated_at, strategy):
        super().__init__(initiated_at, strategy)

    def has_breached_start(self):
        up_breach = self.direction == UP and (self.strategy.ticks.iloc[-1].last_price <= self.strategy.ticks.loc[self.initiated_at].last_price).all()
        if up_breach:
            return up_breach
        down_breach = self.direction == DOWN and (self.strategy.ticks.iloc[-1].last_price >= self.strategy.ticks.loc[self.initiated_at].last_price).all()
        return down_breach

    def get_new_phase_start(self):
        """
        Return new phase start when current phase is breached
        """
        if self.direction == UP:
            return self.strategy.ticks.loc[self.strategy.ticks.index > self.initiated_at].last_price.idxmin()
        else:
            return self.strategy.ticks.loc[self.strategy.ticks.index > self.initiated_at].last_price.idxmax()

    def next(self, tick):
        if self.status == self.STATUS_TERM:
            raise Exception(f"phase {self} already terminated")
        if self.status == self.STATUS_INITIATED:
            if self.strategy.ticks.iloc[-1].ch > 0:
                self.direction = UP
            else:
                self.direction = DOWN
            self.status = self.STATUS_CONFIRMED
            return
        if self.has_breached_start():
            self.status = self.STATUS_TERM
            self.term_at = self.strategy.ticks.iloc[-1].name
            self.strategy.on_termination()
            return


class PhaseStrategy(b.BasePhaseStrategy):
    def __init__(self, instrument: "Instrument", settings: Dict):
        super().__init__(instrument, settings)
        self.om = ut.ZerodhaOrderManager()

    def calculate_data(self):
        self.ticks["ch"] = self.ticks.last_price - self.ticks.last_price.shift(1)
        self.ticks["vwch"] = self.ticks.ch * self.ticks.volume  # Volume weighted change
        
    def next(self, tick: Dict):
        super().next(tick)
        self.calculate_data()
        if self.current_phase is None:
            self.current_phase = Phase(initiated_at=self.ticks.iloc[-1].name, strategy=self)
        else:
            self.current_phase.next(tick=tick)

    def on_termination(self):
        logger.info(f"phase {self} terminated")
        if self.om.has_intrade_orders():
            self.om.square_off_all_orders()
        self.inactive_ph.append(self.current_phase)
        new_start = self.current_phase.get_new_phase_start()
        self.current_phase = Phase(initiated_at=new_start, strategy=self)


ps = PhaseStrategy(instrument=instrument, settings=settings)
# for i in range(tdf.shape[0]):
for i in range(50):
    ps.next(tdf.iloc[i].to_dict())

print(ps.current_phase)
print(ps.inactive_ph)

2024-03-17 18:42:47,945 - INFO - 4204455325.py:56 - [134763960132864] - 
phase <__main__.PhaseStrategy object at 0x7a90e80d0590> terminated
2024-03-17 18:42:47,958 - INFO - base.py:127 - [134763960132864] - 
has_intrade_orders called


Phase:DOWN, CONFIRMED I:3, C:None R:None, T:None, RJ:None
[Phase:DOWN, TERMINATED I:0, C:None R:None, T:3, RJ:None]


In [5]:
from bokeh.models import ColumnDataSource

ticks_cdf = ColumnDataSource(ps.ticks)
ut.bokeh_plot(cds=ticks_cdf, x_label="timestamp", y_label="price", plot='line')


Loading BokehJS ...